<a href="https://colab.research.google.com/github/ryogrid/ryogridJupyterNotebooks/blob/master/llamaindex_qa_model_only_use_openapi_token_blank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/gpt-index-trial

/content/drive/MyDrive/gpt-index-trial


In [ ]:
!cat requirements.txt

llama-index
langchain
transformers
sentence-transformers
sentencepiece

In [ ]:
!pip install -r requirements.txt
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from llama_index import LLMPredictor
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any

In [ ]:
DEFAULT_PROMPT = """
文脈情報は以下です。
---
{context_str}
---
事前知識ではなく、文脈情報を参考に質問に答えてください。：{query_str}
"""

REFINE_PROMPT = """
質問は以下です。：{query_str}
すでに答えの候補があります。：{existing_answer}
必要な場合のみ、以下の文脈情報を使ってこの答えを改良することができます。
---
{context_msg}
---
この文脈情報により、元の答えを改良して質問に答えてください。
文脈情報が有用でない場合は元の答えをそのまま返してください。
"""

In [ ]:
import torch
from transformers import AutoTokenizer, LukeForQuestionAnswering, T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt-1b")
qa_model = AutoModelForCausalLM.from_pretrained("oshizo/qa-refine-japanese-gpt-1b")

#if torch.cuda.is_available():
#    qa_model = qa_model.to("cuda")


In [ ]:
import openai

openai.api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

def qa_with_openai(question, context):
  prompt = f"""
質問: {question}
文脈情報は以下です。
---
{context}
---
事前知識ではなく、文脈情報を参考に質問に答えてください。
"""

  response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role": "user", "content": prompt},
      ],
  )
  return response.choices[0]["message"]["content"].strip()

In [ ]:
def generate(prompt):

    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    n = len(token_ids[0])

    with torch.no_grad():
        output_ids = qa_model.generate(
            token_ids.to(qa_model.device),
            max_length=n+100,
            min_length=n+2,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    output = tokenizer.decode(output_ids.tolist()[0][n:])
    return output.replace("</s>", "")

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding

# load in HF embedding model from langchain
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="oshizo/sbert-jsnli-luke-japanese-base-lite"))
embed_model._langchain_embedding.client.max_seq_length = 256

In [ ]:
class CustomLLM(LLM):

    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return generate(prompt)
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name":"custom"}
llm = CustomLLM()
llm_predictor = LLMPredictor(llm=llm)

In [ ]:
import urllib
import json
url = "https://ja.wikipedia.org/wiki/%E3%81%BC%E3%81%A3%E3%81%A1%E3%83%BB%E3%81%96%E3%83%BB%E3%82%8D%E3%81%A3%E3%81%8F!?action=cirrusdump"
with urllib.request.urlopen(url) as f:
    data = f.read()
text = json.loads(data)[0]["_source"]["text"]

In [ ]:
from typing import Dict
from llama_index.callbacks.base import BaseCallbackHandler
from llama_index.callbacks.schema import CBEventType

class CustomCallbackHandler(BaseCallbackHandler):

    def __init__(
        self,
        event_starts_to_ignore: Optional[List[CBEventType]] = None,
        event_ends_to_ignore: Optional[List[CBEventType]] = None,
        print_trace_on_end: bool = True,
    ) -> None:
        event_starts_to_ignore = event_starts_to_ignore if event_starts_to_ignore else []
        event_ends_to_ignore = event_ends_to_ignore if event_ends_to_ignore else []
        super().__init__(
            event_starts_to_ignore=event_starts_to_ignore,
            event_ends_to_ignore=event_ends_to_ignore,
        )

    def on_event_start(
        self,
        event_type: CBEventType,
        payload: Optional[Dict[str, Any]] = None,
        event_id: str = "",
        **kwargs: Any
    ) -> str:
        print(f"event_type = {event_type} (start)")
        return event_id

    def on_event_end(
        self,
        event_type: CBEventType,
        payload: Optional[Dict[str, Any]] = None,
        event_id: str = "",
        **kwargs: Any
    ) -> None:
        print(f"event_type = {event_type} (end)")
        if event_type == CBEventType.RETRIEVE:
          for selected_node in payload["nodes"]:
            print(selected_node)

    def start_trace(self, trace_id: Optional[str] = None) -> None:
        print("start_trace")

    def end_trace(
        self,
        trace_id: Optional[str] = None,
        trace_map: Optional[Dict[str, List[str]]] = None,
    ) -> None:
        print("end_trace")

In [ ]:
from llama_index import LLMPredictor, ServiceContext, GPTSimpleKeywordTableIndex, SimpleKeywordTableIndex, VectorStoreIndex
from llama_index.readers.schema.base import Document
from llama_index.indices.list.base import ListRetrieverMode
from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.indices.service_context import set_global_service_context
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.query.response_synthesis import ResponseSynthesizer
from llama_index.indices.response.type import ResponseMode
from llama_index.indices.prompt_helper import PromptHelper

documents = []
for i in range(0, len(text), 200):
    documents.append(Document(text[i:i+200]))
    if i != 0:
        documents.append(Document(text[i-100:i+100]))

llama_debug_handler = LlamaDebugHandler()
custom_callback_handler = CustomCallbackHandler()
callback_manager = CallbackManager([
    llama_debug_handler
    , custom_callback_handler
])

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, callback_manager=callback_manager, prompt_helper=PromptHelper.from_llm_metadata(llm_predictor.get_llm_metadata()), embed_model=embed_model)

# インデックスの作成
index = VectorStoreIndex.from_documents(documents, service_context=service_context, llm_predictor=llm_predictor, embed_model=embed_model)

start_trace
event_type = node_parsing (start)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (start)
event_type = chunking (end)
event_type = chunking (sta

In [ ]:
def get_context():
  contextStr = ""
  num = 0
  print(llama_debug_handler.get_event_pairs(CBEventType.RETRIEVE))
  print(len(llama_debug_handler.get_event_pairs(CBEventType.RETRIEVE)))
  for node in llama_debug_handler.get_event_pairs(CBEventType.RETRIEVE)[0][1].payload["nodes"]:
    contextStr += node.node.text
    num += 1
  print(num)
  return contextStr

In [ ]:
len(documents)

255

In [ ]:
from llama_index.prompts.base import Prompt
from llama_index.prompts.prompts import RefinePrompt, QuestionAnswerPrompti

refine_prompt = RefinePrompt(REFINE_PROMPT)
default_prompt = QuestionAnswerPrompt(DEFAULT_PROMPT)

In [ ]:
retriever = index.as_retriever(retriever_mode='embedding',  similarity_top_k=15)
#retriever = index.as_retriever(retriever_mode='simple')
#retriever = index.as_retriever()
#retriever = index.as_retriever(retriever_mode='embedding')

#response_synthesizer = ResponseSynthesizer.from_args(response_mode = ResponseMode.REFINE ,service_context=service_context, text_qa_template=default_prompt, refine_template=refine_prompt, callback_manager=callback_manager, verbose=True)
#response_synthesizer = ResponseSynthesizer.from_args(response_mode = ResponseMode.COMPACT ,service_context=service_context, text_qa_template=default_prompt, refine_template=refine_prompt, callback_manager=callback_manager, verbose=True)
response_synthesizer = ResponseSynthesizer.from_args(response_mode = ResponseMode.SIMPLE_SUMMARIZE, service_context=service_context, text_qa_template=default_prompt, callback_manager=callback_manager, verbose=True)
query_engine = RetrieverQueryEngine(retriever=retriever, response_synthesizer=response_synthesizer, callback_manager=callback_manager)

In [ ]:
response = query_engine.query(
    "虹夏ちゃんのお姉さんの仕事は？"
)
print(response.response)
print(get_context())
print(qa_with_openai("虹夏ちゃんのお姉さんの仕事は？", get_context()))


start_trace
event_type = query (start)
event_type = retrieve (start)
event_type = embedding (start)
event_type = embedding (end)
event_type = retrieve (end)
NodeWithScore(node=Node(text='へと呼び方を改める。同じ学校のひとりのことを気にかけるが、一方でひとりの性格などを歯に衣着せず述べ、無自覚のうちに精神的ダメージを与えることもしばしば。そのストレートな物言いは虹夏（アニメではリョウ）に「ナチュラルに鬼畜」と評された。両親は共に公務員であり、曰く「母親はしっかりしている」とのこと。 伊地知 星歌（いじち せいか） 声 - 内田真礼 虹夏の姉で、ライブハウス「STARRY」の店長。ク', doc_id='3bbe9871-d8a8-4a1e-98f1-ed7b335d6301', embedding=None, doc_hash='f1fffd144b29d844d5944e7ddbd128d643c1dca420f521ff45122e4c0474b2cf', extra_info=None, node_info={'start': 0, 'end': 200, '_node_type': <NodeType.TEXT: '1'>}, relationships={<DocumentRelationship.SOURCE: '1'>: '40e7e2c6-d020-4f98-ab69-a3a30f3521b5'}), score=0.6322367996182165)
NodeWithScore(node=Node(text='とは公にはされていない。 伊地知 虹夏（いじち にじか） 声 - 鈴代紗弓 誕生日：5月29日 / 身長：154cm / 体重：48kg / 血液型：A型 ドラム担当。下北沢高校2年→3年。明るく世話焼きなバンドのリーダーでまとめ役。また、本作におけるツッコミ役。水玉のリボンがトレードマークで、たいてい身体のどこかに身につけている。姉の星歌がライブハウスの店長を務める元バンドマンであることの影響か', doc_id='dff77621-9b32-4eeb-91

In [ ]:
response = query_engine.query(
    "後藤ひとりがギターに熱中するようになった理由になった出来事は？",
)

print(response.response)
print(qa_with_openai("後藤ひとりがギターに熱中するようになった理由になった出来事は？", get_context()))

start_trace
event_type = query (start)
event_type = retrieve (start)
event_type = embedding (start)
event_type = embedding (end)
event_type = retrieve (end)
NodeWithScore(node=Node(text='ているといち早く気付いている。 先述の性格に加え、運動や勉強など取り柄と言えるものがないというコンプレックスを持っていたところ、中学1年時に暗い学生時代から一転してスターとなったバンドマンのインタビューを目にしたことで、父親の直樹から借りたギターに没頭する。高校入学後はギターを持って公園にいたところを虹夏から声を掛けられ、逃げた郁代の代わりとして臨時で結束バンドのギターを務める運びになり、その流れ', doc_id='bdea203e-af57-4e97-9d47-c69323e3f30c', embedding=None, doc_hash='149fceaef1e0db6a81f03be82262eee8185340c5c910961eaa382a3ac38104df', extra_info=None, node_info={'start': 0, 'end': 200, '_node_type': <NodeType.TEXT: '1'>}, relationships={<DocumentRelationship.SOURCE: '1'>: '7ce1ca14-a30d-45bf-adb7-51cdf92a76b8'}), score=0.7694375794712924)
NodeWithScore(node=Node(text='島淳司 誕生日：10月16日 / 血液型：O型 後藤姉妹の父。洋楽趣味でギターを所持しており、ひとりは当初そのギターを借りて使用していた。ひとりの音楽活動については「洋楽にハマってほしかった」としながらも好意的に見守っており、文化祭ライブでギターを壊したことを告白された際には「ライブ中に壊すなんて興奮しちゃうなあ」と話すなどロックに対する理解も高いが、当時中1だったひとりに貸し与えたギターは、虹夏', doc_id='f5169ed2-2ae0-4754-ab

In [ ]:
response = query_engine.query(
    "ギターヒーローの正体は誰？",
)
print(response.response)
print(qa_with_openai("ギターヒーローの正体は誰？", get_context()))

start_trace
event_type = query (start)
event_type = retrieve (start)
event_type = embedding (start)
event_type = embedding (end)
event_type = retrieve (end)
NodeWithScore(node=Node(text='女子学生服などを鑑賞用に所持していたりする。また涙もろい。 ひとりのギターヒーローとしての正体まではわからなかったものの、ひとりのギタリストとしての才能にもいち早く気づいており、ひとりを中心にした結束バンドの動画を密かに撮影するなど注目している。 PAさん 声 - 小岩井ことり ミステリアスな風貌をしたSTARRYの女性PAエンジニア。最終学歴は高校中退。プライベートでは「音戯 アルト（おとぎ ア', doc_id='911cacdf-de92-4e97-809d-3621f6d779fe', embedding=None, doc_hash='407f9383e466c6fb2c2398b375714487f035eb13ab23ab0a795db6783e8afab3', extra_info=None, node_info={'start': 0, 'end': 200, '_node_type': <NodeType.TEXT: '1'>}, relationships={<DocumentRelationship.SOURCE: '1'>: '53536030-17dd-48fe-9348-f1c00c9cd676'}), score=0.6518938883814517)
NodeWithScore(node=Node(text='束バンドの取材にSTARRYを訪れ、演奏の癖からひとりとギターヒーローが同一人物であることを見抜く。ギターヒーローの動画に注目する1人で、その実力を既にプロでも通用すると評価している。一方で結束バンドについては「高校生にしたらレベルはまぁ高いと思う」としつつも、その活動姿勢を「ガチじゃない」「本気でプロを目指しているバンドに見えない」と評し「こんなところでうだうだやってると才能腐っちゃいますよ」と', doc_id='7afa7f63-ce56-49f8-b1

In [ ]:
response = query_engine.query(
    "喜多郁代の両親の仕事は？",
)
print(response.response)
print(qa_with_openai("喜多郁代の両親の仕事は？", get_context()))

start_trace
event_type = query (start)
event_type = retrieve (start)
event_type = embedding (start)
event_type = embedding (end)
event_type = retrieve (end)
NodeWithScore(node=Node(text=' 声 - 末柄里恵 誕生日：4月3日 /血液型：A型 後藤姉妹の母。ひとり及びふたりと同じくピンク色の髪をしている。人見知りなひとりを何かと心配している。喜多が家に来たときは場を盛り上げようと、喜んで学生コスプレまでした（21年ぶりに制服を着たとのこと）。結束バンドを夫と一緒になって応援しており、そのためにも学生に扮したことがある。 後藤 ふたり（ごとう ふたり） 声 - 和多田美咲 誕生日：7月', doc_id='8b170908-ecf5-44c0-89c2-41c576fd2177', embedding=None, doc_hash='13f8ed649c5bd8da206e9c2fee94596210a60d1b1bc43d80ddd53dd224a680d4', extra_info=None, node_info={'start': 0, 'end': 200, '_node_type': <NodeType.TEXT: '1'>}, relationships={<DocumentRelationship.SOURCE: '1'>: '16a693ab-13fd-4575-8300-76c9d63d2969'}), score=0.6503665888554038)
NodeWithScore(node=Node(text='たお金を与えている。 目はいつも何か（髪の毛や、ふたりの手など）に隠されていて描かれていない。 会社では窓際族と説明しているが、自宅は金沢八景（横浜市）の戸建てである。 後藤 美智代（ごとう みちよ） 声 - 末柄里恵 誕生日：4月3日 /血液型：A型 後藤姉妹の母。ひとり及びふたりと同じくピンク色の髪をしている。人見知りなひとりを何かと心配している。喜多が家に来たときは場を盛り上げようと、喜んで', doc_id='4ecf775e-d814-4a0a-b6

In [ ]:
response = query_engine.query(
    "ひとりが動画投稿する時のハンドルネームは？",
)
print(response.response)
qa_with_openai("ひとりが動画投稿する時のハンドルネームは？", get_context())

start_trace
event_type = query (start)
event_type = retrieve (start)
event_type = embedding (start)
event_type = embedding (end)
event_type = retrieve (end)
NodeWithScore(node=Node(text='の子だったが、担当編集から「もっとガチにした方がいい」と指摘を受けて修正していき、たちまち現在の設定となった。 ギターヒーロー ひとりが動画投稿サイトで活動する際に用いるハンドルネーム。ひとりが普段人前に出せないが内面に抱えている承認欲求の象徴とも言えるもので、現実のひとりからはかけ離れた「バスケ部エースの彼氏持ち」や「ラインの友達数は1000人超え」といった設定がつけられている。顔を隠すアングル', doc_id='f5cf68af-01ac-4e2e-bb14-c46fd679f879', embedding=None, doc_hash='4ad06326b3a986a3f99560bcd630c1d0406a4606d3c849bae1ea563d009ef828', extra_info=None, node_info={'start': 0, 'end': 200, '_node_type': <NodeType.TEXT: '1'>}, relationships={<DocumentRelationship.SOURCE: '1'>: 'ebb03355-346b-4b2e-a663-0104f9b7f5ea'}), score=0.7022262274203019)
NodeWithScore(node=Node(text='ると、顔を上げ表情も整えればアイドル事務所も狙えるほどのルックスだが、その顔は10秒と持たない。そして自己評価が極めて低く、ひとり自身は外見やスタイルを全く意識していない。 初期設定では普通の可愛い女の子だったが、担当編集から「もっとガチにした方がいい」と指摘を受けて修正していき、たちまち現在の設定となった。 ギターヒーロー ひとりが動画投稿サイトで活動する際に用いるハンドルネーム。ひとりが普段人', doc_id='7a370515-f540-4129-87

'質問の文脈からは、ひとりが動画投稿する際のハンドルネームについては言及されていません。'

In [ ]:
response = query_engine.query(
    "伊地知虹夏の誕生日はいつ？",
)
print(response.response)
qa_with_openai("伊地知虹夏の誕生日はいつ？", get_context())


start_trace
event_type = query (start)
event_type = retrieve (start)
event_type = embedding (start)
event_type = embedding (end)
event_type = retrieve (end)
NodeWithScore(node=Node(text='は虹夏の希望（未定）で彼女は作詞作曲には関わっていないため、実際のジャンルは不明である。 ^ 後藤正文の12月2日→後藤ひとりの2月21日、伊地知潔の9月25日→伊地知虹夏の5月29日、山田貴洋の8月19日→山田リョウの9月18日、喜多建介の1月24日→喜多郁代の4月21日。 ^ 本人は他人から初めて名付けてもらったあだ名と喜び、気に入っている。この名を付けられたきっかけは、ライブでのメンバー紹介', doc_id='ca63d239-cdb0-47f3-a25f-a348bf7cb01b', embedding=None, doc_hash='94102736536108bb2f94892f07f32cc8d2bc7e11ad43bb55aa69dc8d95ecea43', extra_info=None, node_info={'start': 0, 'end': 200, '_node_type': <NodeType.TEXT: '1'>}, relationships={<DocumentRelationship.SOURCE: '1'>: 'eafe771c-47c5-457d-b04c-6d48fa0e8783'}), score=0.6173370159664309)
NodeWithScore(node=Node(text='19日→山田リョウの9月18日、喜多建介の1月24日→喜多郁代の4月21日。 ^ 本人は他人から初めて名付けてもらったあだ名と喜び、気に入っている。この名を付けられたきっかけは、ライブでのメンバー紹介時に本名を出してほしくないと希望したため。以前は虹夏から「ひとりちゃん」、リョウから「ひとり」と呼ばれていた。きくりもニックネームを知るまでは「ひとりちゃん」と呼んでいた。 ^ テレビ番組『69号室の', doc_id='45f20b70-4354-4b10-bc

'伊地知虹夏の誕生日は5月29日です。'